# Initialize database tables

## Control table

In [38]:
import os
import numpy as np

from grizli.aws import db
from grizli import utils

import djapipes

In [4]:
djapipes.run.run_pipes_on_dja_spec?

Signature:
djapipes.run.run_pipes_on_dja_spec(
    spec_name,
    sfh='continuity',
    n_age_bins=10,
    scale_disp=1.3,
    dust_type='kriek',
    use_msa_resamp=False,
    fit_agn=False,
    fit_dla=False,
    fit_mlpoly=False,
)
Docstring:
Runs bagpipes on spectrum from DJA AWS database, saves posteriors as .h5 files and plots as .pdf files

Parameters
----------
spec_name : DJA spectrum name, format=str

Returns
-------
None
File:      ~/miniconda3/envs/py310-djapipes/lib/python3.10/site-packages/djapipes/run.py
Type:      function

In [31]:
import time
row = dict(
    file_spec = "rubies-egs61-v3_prism-clear_4233_42328.spec.fits",
    sfh='continuity',
    n_age_bins=10,
    scale_disp=1.3,
    dust_type='kriek',
    use_msa_resamp=False,
    fit_agn=False,
    fit_dla=False,
    fit_mlpoly=False,
    status=70,
    ctime=time.time(),
)

tab = utils.GTable([row])
tab

file_spec,sfh,n_age_bins,scale_disp,dust_type,use_msa_resamp,fit_agn,fit_dla,fit_mlpoly,status,ctime
str48,str10,int64,float64,str5,bool,bool,bool,bool,int64,float64
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,continuity,10,1.3,kriek,False,False,False,False,70,1734393587.8455942


In [33]:
if (0):
    # db.execute("DROP TABLE nirspec_bagpipes")
    db.send_to_database("nirspec_bagpipes", tab, if_exists="replace")
    db.execute("CREATE INDEX on nirspec_bagpipes (file_spec)")


db.SQL("select * from nirspec_bagpipes")

file_spec,sfh,n_age_bins,scale_disp,dust_type,use_msa_resamp,fit_agn,fit_dla,fit_mlpoly,status,ctime
str48,str10,int64,float64,str5,bool,bool,bool,bool,int64,float64
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,continuity,10,1.3,kriek,False,False,False,False,70,1734393587.8455942


## Results table

In [45]:
# ! ls pipes/cats/rubies-egs61-v3_prism-clear_4233_42328/
post = utils.read_catalog("pipes/cats/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_postcat.csv")
post["sfh"] = "continuity"
post["dust_type"] = "kriek"
for c in post.colnames:
    if post[c].dtype == np.float64:
        post[c] = post[c].astype(np.float32)

if (0):
    db.send_to_database("nirspec_bagpipes_posterior", post, if_exists="replace")
    db.execute("CREATE INDEX on nirspec_bagpipes_posterior (file_spec)")

db.SQL("select * from nirspec_bagpipes_posterior")

file_spec,file_phot,file_zout,id_phot,dr,filt_num,calib_0_16,calib_0_50,calib_0_84,calib_1_16,calib_1_50,calib_1_84,calib_2_16,calib_2_50,calib_2_84,continuity_dsfr1_16,continuity_dsfr1_50,continuity_dsfr1_84,continuity_dsfr2_16,continuity_dsfr2_50,continuity_dsfr2_84,continuity_dsfr3_16,continuity_dsfr3_50,continuity_dsfr3_84,continuity_dsfr4_16,continuity_dsfr4_50,continuity_dsfr4_84,continuity_dsfr5_16,continuity_dsfr5_50,continuity_dsfr5_84,continuity_dsfr6_16,continuity_dsfr6_50,continuity_dsfr6_84,continuity_dsfr7_16,continuity_dsfr7_50,continuity_dsfr7_84,continuity_dsfr8_16,continuity_dsfr8_50,continuity_dsfr8_84,continuity_massformed_16,continuity_massformed_50,continuity_massformed_84,continuity_metallicity_16,continuity_metallicity_50,continuity_metallicity_84,dust_Av_16,dust_Av_50,dust_Av_84,nebular_logU_16,nebular_logU_50,nebular_logU_84,noise_scaling_16,noise_scaling_50,noise_scaling_84,redshift_16,redshift_50,redshift_84,stellar_mass_16,stellar_mass_50,stellar_mass_84,formed_mass_16,formed_mass_50,formed_mass_84,sfr_16,sfr_50,sfr_84,ssfr_16,ssfr_50,ssfr_84,nsfr_16,nsfr_50,nsfr_84,mass_weighted_age_16,mass_weighted_age_50,mass_weighted_age_84,tform_16,tform_50,tform_84,tquench_16,tquench_50,tquench_84,mass_weighted_zmet_16,mass_weighted_zmet_50,mass_weighted_zmet_84,sfh_16,sfh_50,sfh_84,chisq_phot_16,chisq_phot_50,chisq_phot_84,restU_16,restU_50,restU_84,restV_16,restV_50,restV_84,restJ_16,restJ_50,restJ_84,sfh,dust_type
str48,str36,str43,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str10,str5
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,ceers-full-grizli-v7.4-fix_phot.fits,ceers-full-grizli-v7.4-fix.eazypy.zout.fits,45984,0.0025837352,7,1.0906843,1.1007513,1.111982,0.13404463,0.14948104,0.16724166,-0.025392357,-0.01450396,-0.005704524,1.5992963,2.3499546,3.897448,-5.432896,-2.759181,-1.5772108,-2.6653802,-0.920001,0.101954564,-2.2580905,-0.7151477,0.3022016,-1.9315832,-0.53638446,0.43751964,-1.5627255,-0.2930139,0.77155346,-1.3798062,-0.2493546,0.7266544,-1.347817,-0.22493288,0.79783887,10.594977,10.6036625,10.613153,1.9617285,2.0898528,2.2323523,0.26661077,0.31436917,0.3580495,-3.4096978,-2.4284744,-1.4446774,1.6471881,1.7081825,1.7717766,2.6793857,2.6820676,2.6856956,10.358072,10.366877,10.377017,10.594977,10.6036625,10.613153,3.2800497e-08,3.7763068e-05,0.003439571,-17.849339,-14.788182,-12.8288145,-8.708921,-5.647402,-3.6870823,1.2075636,1.2132057,1.2291875,1.158053,1.1736107,1.1806157,1.4938755,1.497463,1.5014544,1.9557912,2.0847487,2.228576,0.0,1.2216088e-05,0.032659084,13.353081,14.23861,15.93904,4.925325,4.945362,4.966345,2.97721,2.9887047,3.0007243,1.7958202,1.8099265,1.8247521,continuity,kriek


# Sync example

In [47]:
sfh = "continuity"
dust_type = "kriek"

file_spec = "rubies-egs61-v3_prism-clear_4233_42328.spec.fits"


In [62]:
import glob

runName = file_spec.split('.spec.fits')[0]
root = file_spec.split('-v3')[0] + '-v3'

s3_path = f"s3://msaexp-nirspec/pipes/{root}"

files = glob.glob(f"pipes/cats/{runName}/*")
files += glob.glob(f"pipes/plots/{runName}/*")
files += glob.glob(f"pipes/posterior/{runName}/*")

for file in files:
    ! aws s3 cp {file} {s3_path}/ --acl public-read


upload: pipes/cats/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_calibcurve.csv to s3://msaexp-nirspec/pipes/rubies-egs61-v3/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_calibcurve.csv
upload: pipes/cats/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_postcat.csv to s3://msaexp-nirspec/pipes/rubies-egs61-v3/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_postcat.csv
upload: pipes/plots/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_calib.png to s3://msaexp-nirspec/pipes/rubies-egs61-v3/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_calib.png
upload: pipes/plots/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_specfit_flam.png to s3://msaexp-nirspec/pipes/rubies-egs61-v3/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_specfit_flam.png
upload: pipes/plots/rubies-egs

# Scripts on EC2

In [76]:
from importlib import reload
reload(djapipes.run); reload(djapipes.utils)
reload(djapipes.run); reload(djapipes.utils)


<module 'djapipes.utils' from '/Users/gbrammer/miniconda3/envs/py310-djapipes/lib/python3.10/site-packages/djapipes/utils.py'>

In [93]:
#!/usr/bin/env python

# Wrapper around djapipes.run

import os
import glob
import yaml
import warnings
import time

import numpy as np
import matplotlib.pyplot as plt
plt.ioff()

import astropy.time

import grizli
from grizli import utils
utils.set_warnings()

HOME = '/GrizliImaging/'
HOME = os.getcwd()

def get_random_bagpipes_file():
    """
    Get a table row of a file to be processed

    Returns
    -------
    res : None, table row
        Returns a single row from `reprocess_rates` with ``status == 0``, or None if
        nothing found

    """
    from grizli.aws import db

    rows = db.SQL(f"""SELECT *
        FROM nirspec_bagpipes
        WHERE status = 0 order by random() limit 1
    """)

    if len(rows) == 0:
        return None
    else:
        return rows


def run_djapipes_wrapper(file_spec="rubies-egs61-v3_prism-clear_4233_42328.spec.fits", sfh="continuity", dust_type="kriek", **kwargs):
    """
    Wrapper around djapipes.run.run_pipes_on_dja_spec
    """
    import glob
    
    import djapipes
    
    status = 0
    
    try:
        djapipes.run.run_pipes_on_dja_spec(
            file_spec=file_spec,
            sfh=sfh,
            dust_type=dust_type,
            **kwargs
        )
        
        status |= 2
    except:
        status |= 4
        return status

    runName = file_spec.split('.spec.fits')[0]
    root = file_spec.split('-v3')[0] + '-v3'

    s3_path = f"s3://msaexp-nirspec/pipes/{root}"

    files = glob.glob(f"pipes/cats/{runName}/*")
    files += glob.glob(f"pipes/plots/{runName}/*")
    files += glob.glob(f"pipes/posterior/{runName}/*")
    
    if len(files) == 0:
        print(f"No outputs found in pipes/*/{runName}")
        
        status |= 8 
    
    for file in files:
        os.system(f"aws s3 cp {file} {s3_path}/ --acl public-read")
    
    # Posterior result
    post_file = f"pipes/cats/{runName}/{runName}_{sfh}_{dust_type}_postcat.csv"
    if os.path.exists(post_file):
        
        print(f'Send {post_file} to nirspec_bagpipes_posterior')
        
        post = utils.read_catalog(post_file)
        post["sfh"] = sfh
        post["dust_type"] = dust_type
    
        for c in post.colnames:
            if post[c].dtype == np.float64:
                post[c] = post[c].astype(np.float32)
        
        # Remove row if it currently exists
        db.execute(f"""
        DELETE FROM nirspec_bagpipes_posterior
        WHERE file_spec = '{file_spec}'
        AND sfh = '{sfh}'
        AND dust_type = '{dust_type}'
        """)
        
        db.send_to_database("nirspec_bagpipes_posterior", post, if_exists="append")
    else:
        print("posterior summary {post_file} not found")
        status |= 16
    
    return status

    
def run_one():
    """
    Run a file with status = 0
    """
    import os
    import time
    import boto3

    from grizli.aws import db
    from grizli import utils


    row = get_random_bagpipes_file()

    if row is None:
        print('Nothing to do.')
        with open(os.path.join(HOME, 'nirspec_pipes_finished.txt'),'w') as fp:
            fp.write(time.ctime() + '\n')

        return None

    print(f'============  NIRSpec dja-pipes  ==============')
    print(f"{row['file_spec'][0]}")
    print(f'========= {time.ctime()} ==========')

    file_prefix = row['file_spec'][0].split('.spec')[0]
    # key = row['root'][0] + '-' + file_prefix
    key = f"{file_prefix}-pipes"

    WORK_PATH = os.path.join(HOME, key)
    if os.path.exists(WORK_PATH):
        print('! already exists, skip')
        return row
    
    ORIG_PATH = os.getcwd()
    
    os.makedirs(WORK_PATH)    
    os.chdir(WORK_PATH)
    
    with open(os.path.join(HOME, 'nirspec_pipes_history.txt'),'a') as fp:
        fp.write(f"{time.ctime()} {row['file_spec'][0]}\n")

    # Update db
    db.execute(f"""
        UPDATE nirspec_bagpipes
        SET status = 1, ctime={time.time()}
        WHERE
            file_spec = '{row['file_spec'][0]}'
        """
    )

    #################
    # Run it
    #################
    status = run_djapipes_wrapper(**row[0])

    # Update db
    db.execute(f"""
        UPDATE nirspec_bagpipes
        SET
            status = {status},
            ctime = {time.time()}
        WHERE
            file_spec = '{row['file_spec'][0]}'
        """
    )

    os.chdir(ORIG_PATH)

    return row


# if __name__ == "__main__":
#     run_one()




In [84]:
run_djapipes_wrapper()

42328 rubies-egs61-v3_prism-clear_4233_42328
100% [............................................................................] 639360 / 639360
Results loaded from pipes/posterior/rubies-egs61-v3_prism-clear_4233_42328/42328.h5

Fitting not performed as results have already been loaded from pipes/posterior/rubies-egs61-v3_prism-clear_4233_42328/42328.h5. To start over delete this file or change run.

upload: pipes/cats/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_calibcurve.csv to s3://msaexp-nirspec/pipes/rubies-egs61-v3/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_calibcurve.csv
upload: pipes/cats/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_postcat.csv to s3://msaexp-nirspec/pipes/rubies-egs61-v3/rubies-egs61-v3_prism-clear_4233_42328_continuity_kriek_postcat.csv
upload: pipes/plots/rubies-egs61-v3_prism-clear_4233_42328/rubies-egs61-v3_prism-clear_4233_42328_continuity_kr

2

In [86]:
rows = db.SQL("select * from nirspec_bagpipes")
rows

file_spec,sfh,n_age_bins,scale_disp,dust_type,use_msa_resamp,fit_agn,fit_dla,fit_mlpoly,status,ctime
str48,str10,int64,float64,str5,bool,bool,bool,bool,int64,float64
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,continuity,10,1.3,kriek,False,False,False,False,70,1734393587.8455942


In [90]:
# New row
rows = rows[:1]
rows['file_spec'] = "rubies-uds43-v3_prism-clear_4233_31823.spec.fits"
rows['status'] = 0
db.send_to_database("nirspec_bagpipes", rows, if_exists='append')

1

In [94]:
db.SQL("select * from nirspec_bagpipes")

file_spec,sfh,n_age_bins,scale_disp,dust_type,use_msa_resamp,fit_agn,fit_dla,fit_mlpoly,status,ctime
str48,str10,int64,float64,str5,bool,bool,bool,bool,int64,float64
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,continuity,10,1.3,kriek,False,False,False,False,70,1734393587.8455942
rubies-uds43-v3_prism-clear_4233_31823.spec.fits,continuity,10,1.3,kriek,False,False,False,False,0,1734393587.8455942


In [95]:
row = run_one()

============  NIRSpec dja-pipes  ==============
rubies-uds43-v3_prism-clear_4233_31823.spec.fits
========= Tue Dec 17 02:05:53 2024 ==========
31823 rubies-uds43-v3_prism-clear_4233_31823
100% [............................................................................] 642240 / 642240
Bagpipes: fitting object 31823


Completed in 355.6 seconds.

Parameter                          Posterior percentiles
                                16th       50th       84th
----------------------------------------------------------
calib:0                        1.276      1.288      1.300
calib:1                       -0.025     -0.007      0.009
calib:2                       -0.017     -0.009     -0.000
continuity:dsfr1               1.583      2.313      4.074
continuity:dsfr2              -6.489     -3.834     -2.348
continuity:dsfr3              -2.194     -0.725      0.223
continuity:dsfr4              -2.081     -0.623      0.387
continuity:dsfr5              -1.845     -0.497      0.478
con

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "WHERE"
LINE 6:         WHERE
                ^

[SQL: 
        UPDATE nirspec_bagpipes
        SET
            status = 2,
            ctime = 1734397969.853627,
        WHERE
            file_spec = 'rubies-uds43-v3_prism-clear_4233_31823.spec.fits'
        ]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [98]:
# Status changed to 2
db.SQL("select * from nirspec_bagpipes")

file_spec,sfh,n_age_bins,scale_disp,dust_type,use_msa_resamp,fit_agn,fit_dla,fit_mlpoly,status,ctime
str48,str10,int64,float64,str5,bool,bool,bool,bool,int64,float64
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,continuity,10,1.3,kriek,False,False,False,False,70,1734393587.8455942
rubies-uds43-v3_prism-clear_4233_31823.spec.fits,continuity,10,1.3,kriek,False,False,False,False,2,1734397969.853627


In [99]:
# Results are in the posterior table
db.SQL("select * from nirspec_bagpipes_posterior")

file_spec,file_phot,file_zout,id_phot,dr,filt_num,calib_0_16,calib_0_50,calib_0_84,calib_1_16,calib_1_50,calib_1_84,calib_2_16,calib_2_50,calib_2_84,continuity_dsfr1_16,continuity_dsfr1_50,continuity_dsfr1_84,continuity_dsfr2_16,continuity_dsfr2_50,continuity_dsfr2_84,continuity_dsfr3_16,continuity_dsfr3_50,continuity_dsfr3_84,continuity_dsfr4_16,continuity_dsfr4_50,continuity_dsfr4_84,continuity_dsfr5_16,continuity_dsfr5_50,continuity_dsfr5_84,continuity_dsfr6_16,continuity_dsfr6_50,continuity_dsfr6_84,continuity_dsfr7_16,continuity_dsfr7_50,continuity_dsfr7_84,continuity_dsfr8_16,continuity_dsfr8_50,continuity_dsfr8_84,continuity_massformed_16,continuity_massformed_50,continuity_massformed_84,continuity_metallicity_16,continuity_metallicity_50,continuity_metallicity_84,dust_Av_16,dust_Av_50,dust_Av_84,nebular_logU_16,nebular_logU_50,nebular_logU_84,noise_scaling_16,noise_scaling_50,noise_scaling_84,redshift_16,redshift_50,redshift_84,stellar_mass_16,stellar_mass_50,stellar_mass_84,formed_mass_16,formed_mass_50,formed_mass_84,sfr_16,sfr_50,sfr_84,ssfr_16,ssfr_50,ssfr_84,nsfr_16,nsfr_50,nsfr_84,mass_weighted_age_16,mass_weighted_age_50,mass_weighted_age_84,tform_16,tform_50,tform_84,tquench_16,tquench_50,tquench_84,mass_weighted_zmet_16,mass_weighted_zmet_50,mass_weighted_zmet_84,sfh_16,sfh_50,sfh_84,chisq_phot_16,chisq_phot_50,chisq_phot_84,restU_16,restU_50,restU_84,restV_16,restV_50,restV_84,restJ_16,restJ_50,restJ_84,sfh,dust_type
str48,str42,str49,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str10,str5
rubies-egs61-v3_prism-clear_4233_42328.spec.fits,ceers-full-grizli-v7.4-fix_phot.fits,ceers-full-grizli-v7.4-fix.eazypy.zout.fits,45984,0.0025837352,7,1.0893284,1.1005278,1.112314,0.13313806,0.1495488,0.16703269,-0.02429107,-0.014711174,-0.004652707,1.5485095,2.3482134,3.9384093,-5.7592254,-2.8553116,-1.5897121,-2.4677985,-0.8909998,0.076552935,-1.9088826,-0.67037135,0.3234663,-2.110811,-0.5237588,0.43503088,-1.7238318,-0.291716,0.7565929,-1.5724771,-0.35703704,0.62429434,-1.3616204,-0.20204504,0.866415,10.594711,10.603731,10.613332,1.9670614,2.1080544,2.2301939,0.26708212,0.30920303,0.35565826,-3.461587,-2.4680204,-1.4733613,1.6415218,1.701191,1.762709,2.6796997,2.682384,2.6859329,10.357736,10.36713,10.376733,10.594711,10.603731,10.613332,2.5779448e-08,3.0501995e-05,0.003697199,-17.954588,-14.884774,-12.790637,-8.811811,-5.742731,-3.6489553,1.2082012,1.2133465,1.2332609,1.1551006,1.173272,1.1804646,1.4934564,1.4971656,1.5013014,1.9611988,2.103084,2.226434,0.0,1.0006521e-05,0.03199979,13.320983,14.231694,15.94693,4.9259133,4.946481,4.9677243,2.976621,2.9893553,3.0018532,1.7955647,1.8092059,1.8264674,continuity,kriek
rubies-uds43-v3_prism-clear_4233_31823.spec.fits,primer-uds-south-grizli-v7.2-fix_phot.fits,primer-uds-south-grizli-v7.2-fix.eazypy.zout.fits,31823,1.0683849e-08,7,1.2750163,1.2873616,1.3006295,-0.024277417,-0.0069819586,0.008741275,-0.016894482,-0.009070048,-5.0981787e-05,1.53419,2.2769182,4.101171,-6.347813,-3.683133,-2.164358,-2.155918,-0.7631303,0.20011736,-2.0920408,-0.5782702,0.3504977,-1.7171866,-0.4967887,0.526128,-1.7692782,-0.411543,0.6400344,-1.7593238,-0.30391493,0.7695858,-1.2967153,-0.08319812,0.9829348,11.100544,11.108019,11.114423,1.823329,1.918836,2.0125654,0.500041,0.53605044,0.5629376,-3.